<a href="https://colab.research.google.com/github/Mahonry/Telstra-Network-Disruptions/blob/master/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_classif
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
event = pd.read_csv('event_type.csv')
train = pd.read_csv('train.csv')
resource = pd.read_csv('resource_type.csv')
log = pd.read_csv('log_feature.csv')
severity = pd.read_csv('severity_type.csv')
test = pd.read_csv('test.csv')

# Inspeccion de los datasets 

In [ ]:
print('event: ',event.shape)
print('resource: ',resource.shape)
print('log: ',log.shape)
print('severity: ',severity.shape)
print('train: ',train.shape)
print('test: ',test.shape)
print('Total data (train + test):  ', 7381 + 11171)
datasets = [event,resource,log,severity,train,test]

event:  (31170, 2)
resource:  (21076, 2)
log:  (58671, 3)
severity:  (18552, 2)
train:  (7381, 3)
test:  (11171, 2)
Total data (train + test):   18552


In [ ]:
for dataset in datasets:
  dataset.info()
  print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31170 entries, 0 to 31169
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          31170 non-null  int64 
 1   event_type  31170 non-null  object
dtypes: int64(1), object(1)
memory usage: 487.2+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21076 entries, 0 to 21075
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             21076 non-null  int64 
 1   resource_type  21076 non-null  object
dtypes: int64(1), object(1)
memory usage: 329.4+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58671 entries, 0 to 58670
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           58671 non-null  int64 
 1   log_feature  58671 non-null  object
 2   volume       58671 non-null  int64 
dtypes: int64(2), object(1)
mem

In [ ]:
#Distribucion de las clases
100*train.fault_severity.value_counts()/train.fault_severity.value_counts().sum()

0    64.815066
1    25.348869
2     9.836066
Name: fault_severity, dtype: float64

## Even type

In [ ]:
# Distribucion de la variable Event
(100*event.event_type.value_counts()/event.event_type.value_counts().sum()).sort_values(ascending=False)

event_type 11    25.306384
event_type 35    21.222329
event_type 34    19.015079
event_type 15    14.100096
event_type 20     4.677575
event_type 54     2.194418
event_type 13     1.867180
event_type 42     1.533526
event_type 44     1.495027
event_type 23     1.376323
event_type 14     1.058710
event_type 43     0.981713
event_type 22     0.715432
event_type 50     0.494065
event_type 10     0.465191
event_type 21     0.436317
event_type 18     0.234200
event_type 47     0.221367
event_type 26     0.208534
event_type 32     0.202117
event_type 30     0.192493
event_type 45     0.170035
event_type 24     0.147578
event_type 27     0.141161
event_type 29     0.134745
event_type 40     0.128329
event_type 46     0.121912
event_type 2      0.118704
event_type 28     0.102663
event_type 8      0.093038
event_type 6      0.089830
event_type 5      0.083414
event_type 7      0.076997
event_type 3      0.060956
event_type 38     0.060956
event_type 36     0.057748
event_type 39     0.057748
e

In [ ]:
event_1 = pd.get_dummies(event, columns=['event_type']).groupby(['id'], as_index=False).sum()

In [ ]:
event_1['sum_events'] = event_1.iloc[:,1:].sum(axis = 1)

## Log_Feature

In [ ]:
log.log_feature = log.log_feature.str.extract('(\d+)').astype(int)
print((100*log.log_feature.value_counts()/log.log_feature.value_counts().sum()).head(20))
log.log_feature

312    8.977178
232    8.102811
82     5.917745
203    4.811576
313    3.655980
233    3.240102
307    2.721958
54     2.681052
170    2.600944
71     2.580491
315    2.548107
134    2.418571
80     2.277105
235    2.205519
193    1.977127
219    1.963491
68     1.862931
227    1.840773
314    1.619199
201    1.537386
Name: log_feature, dtype: float64


0         68
1         68
2         68
3        172
4         56
        ... 
58666    209
58667     54
58668     62
58669    209
58670     87
Name: log_feature, Length: 58671, dtype: int64

In [ ]:
#LOG
log['Features_bin'] = pd.cut(log.log_feature, 193)
log.drop(['log_feature'], axis = 1, inplace = True)

In [ ]:
log_1 = pd.get_dummies(log, columns=['Features_bin']).groupby(['id'], as_index=False).sum()
#log_1 = pd.get_dummies(log, columns=['log_feature']).groupby(['id'], as_index=False).sum()
log_1['sum_log'] = log_1.iloc[:,2:].sum(axis = 1)

In [ ]:
log_1

,id,volume,"Features_bin_(0.615, 2.995]","Features_bin_(2.995, 4.99]","Features_bin_(4.99, 6.984]","Features_bin_(6.984, 8.979]","Features_bin_(8.979, 10.974]","Features_bin_(10.974, 12.969]","Features_bin_(12.969, 14.964]","Features_bin_(14.964, 16.959]","Features_bin_(16.959, 18.953]","Features_bin_(18.953, 20.948]","Features_bin_(20.948, 22.943]","Features_bin_(22.943, 24.938]","Features_bin_(24.938, 26.933]","Features_bin_(26.933, 28.927]","Features_bin_(28.927, 30.922]","Features_bin_(30.922, 32.917]","Features_bin_(32.917, 34.912]","Features_bin_(34.912, 36.907]","Features_bin_(36.907, 38.902]","Features_bin_(38.902, 40.896]","Features_bin_(40.896, 42.891]","Features_bin_(42.891, 44.886]","Features_bin_(44.886, 46.881]","Features_bin_(46.881, 48.876]","Features_bin_(48.876, 50.87]","Features_bin_(50.87, 52.865]","Features_bin_(52.865, 54.86]","Features_bin_(54.86, 56.855]","Features_bin_(56.855, 58.85]","Features_bin_(58.85, 60.845]","Features_bin_(60.845, 62.839]","Features_bin_(62.839, 64.834]","Features_bin_(64.834, 66.829]","Features_bin_(66.829, 68.824]","Features_bin_(68.824, 70.819]","Features_bin_(70.819, 72.813]","Features_bin_(72.813, 74.808]","Features_bin_(74.808, 76.803]",...,"Features_bin_(308.202, 310.197]","Features_bin_(310.197, 312.192]","Features_bin_(312.192, 314.187]","Features_bin_(314.187, 316.181]","Features_bin_(316.181, 318.176]","Features_bin_(318.176, 320.171]","Features_bin_(320.171, 322.166]","Features_bin_(322.166, 324.161]","Features_bin_(324.161, 326.155]","Features_bin_(326.155, 328.15]","Features_bin_(328.15, 330.145]","Features_bin_(330.145, 332.14]","Features_bin_(332.14, 334.135]","Features_bin_(334.135, 336.13]","Features_bin_(336.13, 338.124]","Features_bin_(338.124, 340.119]","Features_bin_(340.119, 342.114]","Features_bin_(342.114, 344.109]","Features_bin_(344.109, 346.104]","Features_bin_(346.104, 348.098]","Features_bin_(348.098, 350.093]","Features_bin_(350.093, 352.088]","Features_bin_(352.088, 354.083]","Features_bin_(354.083, 356.078]","Features_bin_(356.078, 358.073]","Features_bin_(358.073, 360.067]","Features_bin_(360.067, 362.062]","Features_bin_(362.062, 364.057]","Features_bin_(364.057, 366.052]","Features_bin_(366.052, 368.047]","Features_bin_(368.047, 370.041]","Features_bin_(370.041, 372.036]","Features_bin_(372.036, 374.031]","Features_bin_(374.031, 376.026]","Features_bin_(376.026, 378.021]","Features_bin_(378.021, 380.016]","Features_bin_(380.016, 382.01]","Features_bin_(382.01, 384.005]","Features_bin_(384.005, 386.0]",sum_log
0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,5,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18547,18548,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
18548,18549,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Resource

In [ ]:
#Resource
(100*resource.resource_type.value_counts()/resource.resource_type.value_counts().sum()).sort_index()

resource_type 1      0.275195
resource_type 10     0.346366
resource_type 2     42.313532
resource_type 3      0.687986
resource_type 4      1.565762
resource_type 5      0.066426
resource_type 6      2.761435
resource_type 7      2.362877
resource_type 8     48.718922
resource_type 9      0.901499
Name: resource_type, dtype: float64

In [ ]:
resource_1 = pd.get_dummies(resource, columns=['resource_type']).groupby(['id'], as_index=False).sum()
resource_1['sum_resource'] = resource_1.iloc[:,1:].sum(axis = 1)

In [ ]:
resource_1

,id,resource_type_resource_type 1,resource_type_resource_type 10,resource_type_resource_type 2,resource_type_resource_type 3,resource_type_resource_type 4,resource_type_resource_type 5,resource_type_resource_type 6,resource_type_resource_type 7,resource_type_resource_type 8,resource_type_resource_type 9,sum_resource
0,1,0,0,0,0,0,0,1,0,1,0,2
1,2,0,0,1,0,0,0,0,0,0,0,1
2,3,0,0,0,0,0,0,0,0,1,0,1
3,4,0,0,1,0,0,0,0,0,0,0,1
4,5,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18547,18548,0,0,1,0,0,0,0,0,0,0,1
18548,18549,0,0,1,0,0,0,0,0,0,0,1
18549,18550,0,0,0,0,0,0,0,0,1,0,1
18550,18551,0,0,0,0,0,0,0,0,1,0,1


## Severity

In [ ]:
#severity
(100*severity.severity_type.value_counts()/severity.severity_type.value_counts().sum()).sort_index()

severity_type 1    47.046141
severity_type 2    47.094653
severity_type 3     0.043122
severity_type 4     5.465718
severity_type 5     0.350367
Name: severity_type, dtype: float64

In [ ]:
severity = pd.get_dummies(severity, columns=['severity_type']).groupby(['id'], as_index=False).sum()
severity

,id,severity_type_severity_type 1,severity_type_severity_type 2,severity_type_severity_type 3,severity_type_severity_type 4,severity_type_severity_type 5
0,1,1,0,0,0,0
1,2,0,1,0,0,0
2,3,1,0,0,0,0
3,4,0,0,0,1,0
4,5,0,1,0,0,0
...,...,...,...,...,...,...
18547,18548,0,1,0,0,0
18548,18549,1,0,0,0,0
18549,18550,1,0,0,0,0
18550,18551,1,0,0,0,0


## Locations

In [ ]:
#train.location = train.location.str.extract('(\d+)').astype(int)
#test.location = test.location.str.extract('(\d+)').astype(int)

In [ ]:
#Locations 
(100*train.location.value_counts()/train.location.value_counts().sum())

location 821     1.151605
location 1107    1.056767
location 734     1.016122
location 1008    0.961929
location 126     0.961929
                   ...   
location 987     0.013548
location 947     0.013548
location 254     0.013548
location 117     0.013548
location 289     0.013548
Name: location, Length: 929, dtype: float64

In [ ]:
#Location Count Train:
location_count = train['location'].value_counts()
train['location_count'] = train['location'].apply(lambda x: location_count[x])
#train = pd.get_dummies(train, columns=['location'])

In [ ]:
#Location Count Test:
location_count = test['location'].value_counts()
test['location_count'] = test['location'].apply(lambda x: location_count[x])
#test = pd.get_dummies(test, columns=['location'])


In [ ]:
#Convirtiendo location a numerico:
le_1 = LabelEncoder()
le_2 = LabelEncoder()

train['location'] = le_1.fit_transform(train['location'])
test['location'] = le_2.fit_transform(test['location'])

#Consolidando los dataframes para el algoritmo

In [ ]:
# Consolidamos el dataframe de entrenamiento
frames = [log_1,event_1,resource_1,severity]
train_consolidado = train.copy()
test_consolidado = test.copy()

for frame in frames:
  train_consolidado = train_consolidado.merge(frame, on = ['id'])
  test_consolidado = test_consolidado.merge(frame, on = ['id'])

In [ ]:
train_consolidado.shape

(7381, 269)

# Guardado

In [ ]:
train_consolidado.to_csv('train_2.csv', index = False)

In [ ]:
test_consolidado.to_csv('test_2.csv', index = False)

Consolidado: Se utilizo log['Features_bin'] = pd.qcut(log.log_feature, 10) agrupando log en 10 grupos percentiles

Completo: Se aplico get_dummies a todas las variables sin agrupacion 

_2: Se utili log['Features_bin'] = pd.cut(log.log_feature, 193) para reducir a la mitad la cantidad de features dadas por Log